# App Profile Recommendation

The goal of this project is to find mobile app profiles in the Google Play and Apple Store that are profitable for their respective markets. As a thought experiment, we are playing the roles of data analysts for a company that builds these apps, and our focus is provide software engineers the necessary information to make data-driven decisions on how to make popular, profitable, apps.

Many apps are free to download and install, and our main source of revenue consists of in-app ads. This means our revenue for any given app is mostly influenced by the number of users who use our app — the more users that see and engage with the ads, the better. Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

In [1]:
from csv import reader #downloading necessary packages
# opening apple/google files
apple_file = open('AppleStore.csv', encoding="utf8") 
google_file = open("googleplaystore.csv", encoding="utf8")
# reading opened files
read_apple = reader(apple_file)
read_google = reader(google_file)
# making list of lists using open files
apps_apple = list(read_apple)
apps_google = list(read_google)

In [2]:
# creating an explore dataset function
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

# Exploring the data

Below we explore the first few rows of the data for each data set and determined the number of columns and the numbers of rows in each dataset.

In [3]:
print(explore_data(apps_apple, 0, 2, rows_and_columns = True))

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7198
Number of columns: 16
None


In [4]:
print(explore_data(apps_google, 0, 2, rows_and_columns = True))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13
None


# Isolating appropriate variables

For more information on the variables listed in the Apple file, [click here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps).

For more information on the variables listed in the Google file, [click here](https://www.kaggle.com/lava18/google-play-store-apps).

In [5]:
print("Apple File Column Names:")
print(apps_apple[0])
print("\n")
print("Google File Column Names:") 
print(apps_google[0])
print("\n")

Apple File Column Names:
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Google File Column Names:
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']




# Cleaning the data

As you may recall, we are a company that only make free to download and install and are directed to English-speaking audiences. As a result, we may want to remove apps not in english and apps that are not free to target our analysis.

One of the first things we need to do is correct the error listed in [this discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) on the dataset.

In [6]:
# Checking that all rows have same number of columns as header
nvar_google = len(apps_google[0])
for app in apps_google[1:]:
    len_vars = len(app)
    if len_vars != nvar_google:
        missing_index = apps_google.index(app)
missing_var = apps_google[missing_index]
# Deleting row that is not complete
del apps_google[missing_index]

Next we want to take a look at duplicate data. The google play store seems to have multiple entries for the same app. Our next task is to remove these duplicates and only keep the most-up-to-date entry. To determine the most recent entry, we will assume that the entry with the most reviews are the most recent.

For this section, we will first comb through the data by the app name and see how many duplicates we encounter. We will then examine those duplicates and keep the entry with the highest number of reviews.

In [7]:
# This code confirms that there are duplicate rows
unique = []
duplicates = [] 
for app in apps_google[1:]:
    if app[0] in unique:
        duplicates.append(app[0])
    else:
        unique.append(app[0])
print(len(duplicates))

1181


Next, we create a dictionary of all the unique app names. To their respective names, we are ensuring that each name has their respective maximum number of reviews. This will allow us to match the proper row in the large dataset and filter it out from the other duplicate rows.

We start by combing through the android apps file, and assigning each element's name to a variable `name`. Next, we assign each element's number of reviews to a variable called `n_reviews`. We then use a conditional statement to check if the app's name is in the dictionary `reviews max`. If there is an app with the same name already there, we check and see which app element has the most reviews, and if the new app being checked does, then the max reviews number for that app in the dictionary are changed.

If the app is not in the dictionary, we generate a dictionary item with the `name`:`n_reviews` as `key`:`value` pairs.

Finally, we confirm that the length of the dictionary matches the length of the data set were we to discard the duplicate rows.

In [8]:
reviews_max = {}
for app in apps_google[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
print (len(reviews_max))

9659


In [9]:
google_clean = []
already_added = []
for app in apps_google[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        google_clean.append(app)
        already_added.append(name)
explore_data(google_clean, 0,3, rows_and_columns = True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


# Removing Non-English reviews

Remember we use English for the apps we develop at our company, and we'd like to analyze only the apps that are directed toward an English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience.

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

In [10]:
def ascii_check(a_string):
    strikes = 0
    for i in a_string:
        if ord(i) > 127:
            strikes += 1
    if strikes > 3:
        return False
    else:
        return True
print(ascii_check("Instagram"))
print(ascii_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(ascii_check("Docs To Go™ Free Office Suite") )
print(ascii_check("Instachat 😜") )

True
False
True
True


In [13]:
english_google = []
english_apple = []
for app in google_clean:
    if ascii_check(app[0]) == True:
        english_google.append(app)
for app in apps_apple[1:]:
    if ascii_check(app[1]) == True:
        english_apple.append(app)
        
explore_data(english_google, 0, 2, rows_and_columns = True)
explore_data(english_apple, 0, 2, rows_and_columns = True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9614
Number of columns: 13
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 6183
Number of columns: 16


# Isolating only free to download and install apps
As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

In [18]:
g_free_eng_apps = []
a_free_eng_apps = []
for app in english_google:
    price = app[7]
    if price == '0':
        g_free_eng_apps.append(app)
for app in english_apple:
    price = app[4]
    if price == '0.0':
        a_free_eng_apps.append(app)
print (len(g_free_eng_apps))
print (len(a_free_eng_apps))
    

8864
3222


# Starting the analysis
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we develop it further.
If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

In [23]:
def freq_table(dataset, index):
    f_table = {}
    for app in dataset:
        if app[index] in f_table:
            f_table[app[index]] += 1
        else:
            f_table[app[index]] = 1
    return f_table

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [26]:
display_table(a_free_eng_apps, 11)

Games : 1874
Entertainment : 254
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 84
Utilities : 81
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 51
News : 43
Travel : 40
Finance : 36
Weather : 28
Food & Drink : 26
Reference : 18
Business : 17
Book : 14
Navigation : 6
Medical : 6
Catalogs : 4


In [28]:
display_table(g_free_eng_apps, 1)

FAMILY : 1676
GAME : 862
TOOLS : 750
BUSINESS : 407
LIFESTYLE : 346
PRODUCTIVITY : 345
FINANCE : 328
MEDICAL : 313
SPORTS : 301
PERSONALIZATION : 294
COMMUNICATION : 287
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 190
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 124
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 73
WEATHER : 71
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 55
BEAUTY : 53


In [29]:
display_table(g_free_eng_apps, 9)

Tools : 749
Entertainment : 538
Education : 474
Business : 407
Productivity : 345
Lifestyle : 345
Finance : 328
Medical : 313
Sports : 307
Personalization : 294
Communication : 287
Action : 275
Health & Fitness : 273
Photography : 261
News & Magazines : 248
Social : 236
Travel & Local : 206
Shopping : 199
Books & Reference : 190
Simulation : 181
Dating : 165
Arcade : 164
Video Players & Editors : 157
Casual : 156
Maps & Navigation : 124
Food & Drink : 110
Puzzle : 100
Racing : 88
Role Playing : 83
Libraries & Demo : 83
Auto & Vehicles : 82
Strategy : 81
House & Home : 73
Weather : 71
Events : 63
Adventure : 60
Comics : 54
Beauty : 53
Art & Design : 53
Parenting : 44
Card : 40
Casino : 38
Trivia : 37
Educational;Education : 35
Board : 34
Educational : 33
Education;Education : 30
Word : 23
Casual;Pretend Play : 21
Music : 18
Racing;Action & Adventure : 15
Puzzle;Brain Games : 15
Entertainment;Music & Video : 15
Casual;Brain Games : 12
Casual;Action & Adventure : 12
Arcade;Action & Advent

In [39]:
f_table_a = freq_table(a_free_eng_apps, 11)

for genre in f_table_a: # stores the sum of user ratings
    total = 0
    len_genre = 0 #stores number of apps specific to each genre
    for app in a_free_eng_apps:
        genre_app = app[11]
        if genre_app == genre:
            num = float(app[5])
            total += num
            len_genre += 1
    avg_ratings = total/len_genre
    print (genre,':', avg_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


In [38]:
f_table_g = freq_table(g_free_eng_apps, 1)

for category in f_table_g: # stores the sum of user ratings
    total = 0
    len_category = 0 #stores number of apps specific to each genre
    for app in g_free_eng_apps:
        category_app = app[1]
        if category_app == category:
            installs = app[5]
            installs = installs.replace('+','')
            installs = installs.replace(',', '')
            installs = float(installs)
            total += installs
            len_category += 1
    avg_installs = total / len_category
    print (category, ':', avg_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_